### Scraping NUFORC sightnings

To be honest this is not great code. I am providing this script only for the sake of accuracy, however it takes several hours to actually parse all the relevant years and extract data, especially because I am also accessing full comments and not only summaries. This could probably be done in tons of different and more efficient ways. That said: it works! 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#Getting initial page and creating an html parser with BS4
url = 'http://www.nuforc.org/webreports/ndxevent.html'
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, 'html.parser')

In [3]:
soup

<html>
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
 <html> <head> <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/> <meta content="Mozilla/4.5b2 [en] (WinNT; I) [Netscape]" name="GENERATOR"/> <title>ndxLocOut</title> </head> <body> <font face="Arial,Helvetica"><font size="+1">National UFO Reporting
Center</font></font> <br/><font face="Arial,Helvetica"><font size="+1">Report Index by Month</font></font> <br/><i>Click on links for details</i> <p><a href="http://www.nwlink.com/~ufocntr">NUFORC Home</a> <br/> 
</p></body> </html>
</head>
<body>
<table cellspacing="1">
<thead>
<tr>
<th bgcolor="#c0c0c0" bordercolor="#000000"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">Reports</font></th>
<th bgcolor="#c0c0c0" bordercolor="#000000"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">Count</font></th>
</tr>
</thead>
<tbody>
<tr valign="TOP">
<td><font color

In [4]:
# Generic function to extract the current table and parse it with BS4
# try/except to catch request errors on urls
def newSoup(element, url, msg):
    new_url = url + str(element.get('href'))
    try:
        response = requests.get(new_url)
        new_html = response.content
        soup = BeautifulSoup(new_html, 'html.parser')
        currentTable = soup.find('tbody')
    except requests.packages.urllib3.exceptions.MaxRetryError as e:
        print(e)
    
    return currentTable

In [41]:
# This loop is parsing all the main table rows on the nuforc webreports page. I am filtering only interesting years
# In my case between 2017 – 2019. I am not creating yearly reports but one unique file with all entries.
table = soup.find('tbody')
base_url = 'http://www.nuforc.org/webreports/'
# Only recent years
Currentyear = ['2017', '2018', '2019', '2020']
data = []
for row in table.find_all('tr'):
    for cell in row.find_all('a'):
        year = cell.contents
        isRecent = any(x in year[0] for x in Currentyear)
#         Filter avoid other years to be taken in consideration
        if isRecent:
            print('start')
            print(year[0])
            yearTable = newSoup(cell, base_url, year[0])
            
            for case in yearTable.find_all('tr'):
                content = case.select('font')
                if content is not None:
                    city = content[1].contents
                    state = content[2].contents
                    shape = content[3].contents
                else:
                    city = ['NaN']
                    state = ['NaN']
                    shape = ['NaN']
#               I need to acess each link to get the full comment and not only a cropped summary
                for link in case.find_all('a'):
                    date = link.contents
                    print(date[0])
                    singleCase = newSoup(link, base_url, date)
                    if singleCase is not None:
                        comment = singleCase.select('font')[1].contents
                    else:
                        comment = ['NaN']
                    data.append([date[0], city[0], state[0], shape[0], comment[0]])
                    
        else:
            print('skipped')

# Creating pandas df with my list of lists.            
df = pd.DataFrame(data, columns=["datetime", "city", "state", "shape", "comment"])
# Preview.
df.head(5)
# Export
df.to_csv('data/allYears_raw.csv', sep=',')

start
12/2020
12/23/20 07:30
12/23/20 03:18
12/23/20 03:00
12/22/20 22:45
12/22/20 22:05
12/22/20 20:00
12/22/20 08:00
12/22/20 06:47
12/21/20 19:10
12/21/20 19:00
12/21/20 18:00
12/21/20 17:30
12/21/20 16:40
12/21/20 16:40
12/21/20 16:40
12/21/20 15:30
12/21/20 03:00
12/21/20 01:40
12/21/20 01:00
12/20/20 00:00
12/20/20 22:30
12/20/20 21:00
12/20/20 17:25
12/20/20 04:06
12/19/20 20:30
12/19/20 20:30
12/19/20 20:14
12/19/20 15:00
12/19/20 13:00
12/19/20 01:00
12/18/20 18:00
12/18/20 16:45
12/18/20 11:45
12/17/20 22:50
12/17/20 21:15
12/17/20 18:58
12/17/20 18:30
12/17/20 16:46
12/17/20 07:00
12/16/20 13:27
12/16/20 13:27
12/16/20 13:27
12/16/20 10:20
12/16/20
12/15/20 20:56
12/15/20 20:35
12/15/20 20:00
12/15/20 19:00
12/15/20 18:25
12/15/20 17:30
12/15/20 11:00
12/14/20 21:30
12/14/20 21:30
12/14/20 11:08
12/13/20 22:20
12/13/20 20:20
12/13/20 20:20
12/13/20 20:05
12/13/20 20:05
12/13/20 07:10
12/13/20 01:30
12/13/20 00:00
12/12/20 21:00
12/12/20 21:00
12/12/20 16:50
12/12/20 14:10
12

11/12/20 18:25
11/12/20 18:20
11/12/20 18:10
11/12/20 17:00
11/12/20 17:00
11/12/20 11:30
11/12/20 08:10
11/12/20 03:45
11/11/20 20:50
11/11/20 18:19
11/11/20 18:00
11/11/20 18:00
11/11/20 16:13
11/11/20 16:13
11/11/20 15:26
11/11/20 15:00
11/11/20 11:11
11/11/20 10:11
11/11/20 06:20
11/11/20 06:15
11/11/20 05:45
11/10/20 23:45
11/10/20 23:43
11/10/20 23:37
11/10/20 22:10
11/10/20 21:00
11/10/20 19:43
11/10/20 18:40
11/10/20 11:54
11/10/20 09:00
11/10/20 08:00
11/10/20 07:35
11/10/20 06:20
11/10/20 06:19
11/10/20 03:00
11/10/20 03:00
11/9/20 23:00
11/9/20 22:14
11/9/20 21:00
11/9/20 20:30
11/9/20 19:25
11/9/20 19:05
11/9/20 19:00
11/9/20 19:00
11/9/20 17:30
11/9/20 17:20
11/9/20 17:15
11/9/20 05:30
11/9/20 05:00
11/9/20 02:25
11/8/20 23:30
11/8/20 23:16
11/8/20 21:48
11/8/20 21:30
11/8/20 21:15
11/8/20 19:25
11/8/20 19:25
11/8/20 19:00
11/8/20 18:44
11/8/20 18:05
11/8/20 14:00
11/8/20 13:00
11/8/20 08:25
11/8/20 05:45
11/8/20 03:00
11/7/20 23:00
11/7/20 20:20
11/7/20 19:12
11/7/20 19:1

10/7/20 20:00
10/7/20 19:45
10/7/20 19:35
10/7/20 19:21
10/7/20 19:00
10/7/20 14:38
10/7/20 09:21
10/7/20 09:13
10/7/20 06:15
10/7/20 05:15
10/7/20 03:14
10/7/20 01:05
10/6/20 23:38
10/6/20 22:12
10/6/20 21:00
10/6/20 20:15
10/6/20 20:00
10/6/20 20:00
10/6/20 19:52
10/6/20 19:35
10/6/20 19:13
10/6/20 19:00
10/6/20 17:00
10/6/20 16:15
10/6/20 14:40
10/6/20 11:50
10/6/20 11:00
10/6/20 06:40
10/6/20 06:15
10/6/20 04:30
10/6/20
10/5/20 00:00
10/5/20 21:05
10/5/20 21:00
10/5/20 21:00
10/5/20 20:57
10/5/20 20:20
10/5/20 20:00
10/5/20 19:03
10/5/20 19:01
10/5/20 18:45
10/5/20 17:00
10/5/20 16:01
10/5/20 15:01
10/5/20 14:21
10/5/20 14:12
10/5/20 10:35
10/5/20 08:00
10/5/20 06:47
10/5/20 06:30
10/5/20 03:10
10/5/20 01:25
10/5/20 00:30
10/4/20 00:00
10/4/20 22:30
10/4/20 22:30
10/4/20 22:15
10/4/20 22:00
10/4/20 22:00
10/4/20 21:30
10/4/20 21:30
10/4/20 20:45
10/4/20 20:42
10/4/20 20:00
10/4/20 19:30
10/4/20 14:08
10/4/20 13:28
10/4/20 06:05
10/4/20 01:00
10/3/20 22:55
10/3/20 22:55
10/3/20 22:3

8/30/20 03:45
8/30/20 01:45
8/30/20 01:30
8/30/20 01:30
8/30/20 01:00
8/30/20 00:00
8/30/20
8/29/20 23:50
8/29/20 23:45
8/29/20 23:15
8/29/20 22:30
8/29/20 22:30
8/29/20 22:00
8/29/20 21:30
8/29/20 20:59
8/29/20 20:50
8/29/20 20:50
8/29/20 20:00
8/29/20 19:46
8/29/20 18:40
8/29/20 18:30
8/29/20 18:30
8/29/20 18:15
8/29/20 18:00
8/29/20 17:46
8/29/20 17:00
8/29/20 15:30
8/29/20 08:30
8/29/20 06:12
8/29/20 03:15
8/29/20 03:00
8/28/20 22:00
8/28/20 20:45
8/28/20 20:30
8/28/20 20:28
8/28/20 20:16
8/28/20 12:00
8/28/20 10:19
8/28/20 08:30
8/28/20 04:25
8/28/20 02:30
8/27/20 23:57
8/27/20 23:53
8/27/20 21:20
8/27/20 21:00
8/27/20 20:47
8/27/20 19:45
8/27/20 19:00
8/27/20 12:25
8/27/20 10:30
8/27/20 07:47
8/27/20 05:55
8/27/20 04:00
8/27/20 03:00
8/27/20 01:30
8/27/20 00:45
8/27/20 00:23
8/27/20 00:00
8/26/20 00:00
8/26/20 23:44
8/26/20 23:00
8/26/20 21:55
8/26/20 21:45
8/26/20 21:05
8/26/20 18:00
8/26/20 11:50
8/26/20 10:35
8/26/20 07:30
8/26/20 07:25
8/26/20 05:20
8/26/20 02:30
8/26/20 01:4

8/4/20 20:19
8/4/20 15:39
8/4/20 11:00
8/4/20 05:15
8/4/20 03:00
8/4/20 01:05
8/4/20 01:00
8/4/20 00:23
8/3/20 23:40
8/3/20 23:30
8/3/20 22:33
8/3/20 21:35
8/3/20 21:05
8/3/20 21:00
8/3/20 20:55
8/3/20 19:40
8/3/20 19:30
8/3/20 18:00
8/3/20 16:00
8/3/20 13:00
8/3/20 11:15
8/3/20 09:00
8/3/20 03:59
8/3/20 03:00
8/3/20 01:00
8/2/20 23:30
8/2/20 23:00
8/2/20 21:45
8/2/20 21:45
8/2/20 21:30
8/2/20 21:30
8/2/20 21:00
8/2/20 17:00
8/2/20 13:43
8/2/20 09:57
8/2/20 07:30
8/2/20 05:00
8/2/20 03:57
8/2/20 03:47
8/2/20 03:22
8/2/20 01:45
8/2/20 00:00
8/1/20 22:50
8/1/20 22:50
8/1/20 22:47
8/1/20 22:30
8/1/20 22:15
8/1/20 21:15
8/1/20 21:00
8/1/20 21:00
8/1/20 20:55
8/1/20 20:41
8/1/20 19:45
8/1/20 19:16
8/1/20 17:50
8/1/20 17:00
8/1/20 14:00
8/1/20 12:45
8/1/20 10:50
8/1/20 10:30
8/1/20 07:00
8/1/20 05:53
8/1/20 04:00
8/1/20 04:00
8/1/20 03:04
8/1/20 01:10
8/1/20 01:00
8/1/20 00:00
start
07/2020
7/31/20 23:15
7/31/20 23:00
7/31/20 22:53
7/31/20 22:30
7/31/20 22:30
7/31/20 22:00
7/31/20 22:00
7/31

...

5/26/17 03:00
5/25/17 22:33
5/25/17 22:30
5/25/17 22:30
5/25/17 21:00
5/25/17 19:15
5/25/17 18:00
5/25/17 04:30
5/25/17 03:00
5/25/17 02:00
5/25/17 02:00
5/24/17 23:15
5/24/17 22:30
5/24/17 22:30
5/24/17 21:40
5/24/17 21:00
5/24/17 07:12
5/24/17 01:30
5/24/17 01:03
5/24/17 00:30
5/23/17 23:27
5/23/17 22:50
5/23/17 22:00
5/23/17 20:50
5/23/17 20:00
5/23/17 19:30
5/23/17 04:45
5/23/17 01:00
5/22/17 23:18
5/22/17 22:45
5/22/17 17:30
5/22/17 11:30
5/22/17 09:45
5/22/17 04:10
5/22/17 04:00
5/22/17 04:00
5/22/17 01:30
5/22/17 00:08
5/21/17 23:48
5/21/17 23:02
5/21/17 22:42
5/21/17 22:30
5/21/17 22:30
5/21/17 22:00
5/21/17 22:00
5/21/17 22:00
5/21/17 21:45
5/21/17 21:41
5/21/17 19:44
5/21/17 13:00
5/21/17 13:00
5/21/17 00:00
5/20/17 23:58
5/20/17 22:30
5/20/17 22:00
5/20/17 21:02
5/20/17 21:00
5/20/17 21:00
5/20/17 21:00
5/20/17 19:00
5/20/17 13:40
5/20/17 12:15
5/20/17 12:00
5/20/17 10:39
5/20/17 08:30
5/19/17 21:20
5/19/17 21:17
5/19/17 17:49
5/19/17 17:00
5/19/17 13:38
5/19/17 10:49
5/19/1

4/11/17 20:40
4/11/17 20:30
4/11/17 20:30
4/11/17 20:24
4/11/17 20:20
4/11/17 20:00
4/11/17 18:00
4/11/17 00:20
4/10/17 00:00
4/10/17 23:30
4/10/17 23:00
4/10/17 22:00
4/10/17 21:40
4/10/17 21:30
4/10/17 21:15
4/10/17 21:06
4/10/17 21:00
4/10/17 21:00
4/10/17 20:50
4/10/17 20:45
4/10/17 19:15
4/10/17 19:00
4/10/17 11:30
4/10/17 09:00
4/10/17 05:00
4/10/17 01:00
4/10/17 00:10
4/10/17
4/9/17 23:00
4/9/17 22:40
4/9/17 22:30
4/9/17 21:00
4/9/17 21:00
4/9/17 17:15
4/9/17 14:05
4/9/17 13:00
4/9/17 06:18
4/9/17 04:55
4/9/17 02:30
4/9/17 02:00
4/9/17 00:45
4/8/17 22:30
4/8/17 22:27
4/8/17 22:00
4/8/17 22:00
4/8/17 21:50
4/8/17 21:10
4/8/17 21:00
4/8/17 20:30
4/8/17 18:16
4/8/17 16:30
4/8/17 15:36
4/8/17 09:51
4/7/17 22:32
4/7/17 22:30
4/7/17 22:25
4/7/17 21:00
4/7/17 21:00
4/7/17 20:52
4/7/17 20:00
4/7/17 19:16
4/7/17 19:00
4/7/17 18:00
4/7/17 16:30
4/7/17 11:50
4/7/17 09:25
4/7/17 05:50
4/7/17 02:37
4/7/17 01:00
4/6/17 21:50
4/6/17 21:35
4/6/17 21:30
4/6/17 19:40
4/6/17 19:00
4/6/17 12:00
4/6

2/20/17 20:10
2/20/17 20:00
2/20/17 19:20
2/20/17 18:40
2/20/17 18:30
2/20/17 12:45
2/20/17 01:32
2/19/17 23:05
2/19/17 23:00
2/19/17 22:50
2/19/17 22:30
2/19/17 22:04
2/19/17 22:00
2/19/17 21:05
2/19/17 20:58
2/19/17 20:30
2/19/17 20:00
2/19/17 19:50
2/19/17 19:30
2/19/17 19:30
2/19/17 17:59
2/19/17 17:45
2/19/17 14:00
2/19/17 12:15
2/19/17 11:00
2/19/17 09:05
2/19/17 07:30
2/19/17 05:15
2/19/17 04:12
2/19/17 04:00
2/18/17 23:00
2/18/17 22:55
2/18/17 22:30
2/18/17 22:00
2/18/17 22:00
2/18/17 21:47
2/18/17 21:00
2/18/17 20:45
2/18/17 20:45
2/18/17 20:15
2/18/17 19:10
2/18/17 19:00
2/18/17 19:00
2/18/17 18:00
2/18/17 18:00
2/18/17 17:00
2/18/17 16:00
2/18/17 06:00
2/18/17 04:00
2/18/17 03:00
2/18/17 00:00
2/17/17 23:50
2/17/17 23:27
2/17/17 23:20
2/17/17 23:00
2/17/17 22:15
2/17/17 20:40
2/17/17 20:30
2/17/17 20:07
2/17/17 20:03
2/17/17 19:00
2/17/17 18:45
2/17/17 18:00
2/17/17 15:20
2/17/17 08:30
2/17/17 01:00
2/16/17 23:30
2/16/17 22:30
2/16/17 21:00
2/16/17 20:45
2/16/17 20:00
2/16/1

1/5/17 06:35
1/4/17 21:34
1/4/17 19:20
1/4/17 17:00
1/4/17 13:00
1/3/17 05:19
1/3/17 02:30
1/2/17 23:50
1/2/17 21:30
1/2/17 20:00
1/2/17 17:30
1/2/17
1/1/17 00:00
1/1/17 22:55
1/1/17 21:00
1/1/17 20:45
1/1/17 20:00
1/1/17 19:45
1/1/17 19:31
1/1/17 19:05
1/1/17 19:00
1/1/17 18:55
1/1/17 18:15
1/1/17 17:45
1/1/17 14:02
1/1/17 13:50
1/1/17 12:30
1/1/17 12:05
1/1/17 12:00
1/1/17 02:30
1/1/17 01:15
1/1/17 01:00
1/1/17 00:45
1/1/17 00:30
1/1/17 00:30
1/1/17 00:20
1/1/17 00:19
1/1/17 00:15
1/1/17 00:15
1/1/17 00:10
1/1/17 00:10
1/1/17 00:10
1/1/17 00:07
1/1/17 00:05
1/1/17 00:01
1/1/17 00:00
1/1/17 00:00
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
ski

,datetime,city,state,shape,comment
0,12/23/20 07:30,Canadian Lakes,MI,Light,"light in sky ,no sound"
1,12/23/20 03:18,Grants,CA,Light,Bright Orb seen below plane level at Mach 1+ S...
2,12/23/20 03:00,East Quogue,NY,Other,"flickering colored lights in the night sky, in..."
3,12/22/20 22:45,Indian River Shores,FL,Changing,Orange orb over Indian River Shores Florida
4,12/22/20 22:05,lake elsinore,CA,Light,saw a glowing object.shape not discerned. trav...


In [ ]:
no